<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz.

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1:Pablo Esteban Zúñiga Varela 
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `https://github.com/PabloZV/Laboratorio-de-Programacion-Cientifica-para-Ciencia-de-Datos`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 23/06/2022
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [7]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
#!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
#!pip install umap-learn

# Librería para NLP
#!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/pablo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/pablo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [9]:
df_comics = pd.read_csv('df_comics.csv')
df_comics_no_label = pd.read_csv('comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [3]:
# queda a labor de su equipo hacer el análisis exploratorio
display(df_comics.head())
display(df_comics.info())

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1285 entries, 0 to 1366
Data columns (total 82 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        1285 non-null   int64  
 1   name                              1285 non-null   object 
 2   real_name                         1164 non-null   object 
 3   full_name                         895 non-null    object 
 4   overall_score                     1285 non-null   object 
 5   history_text                      1285 non-null   object 
 6   powers_text                       961 non-null    object 
 7   intelligence_score                1285 non-null   int64  
 8   strength_score                    1285 non-null   int64  
 9   speed_score                       1285 non-null   int64  
 10  durability_score                  1285 non-null   int64  
 11  power_score                       1285 non-null   int64  
 12  combat

None

## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [4]:
docs = ['The teacher rocks like a good rock & roll','the rock is the best actor in the world']
docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [10]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /home/pablo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'beauti', 'citi']]

In [5]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [11]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,in,is,like,new,rock,roll,teacher,the,world,york
0,1,0,0,0,0,1,0,0,1,0,2,1,1,1,0,0
1,0,1,0,1,0,0,1,1,0,0,1,0,0,3,1,0
2,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [12]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)
print(df.toarray())
pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

[[1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 2 1 0 1 1 1 1 1 0 0 1 0 0 0 0]
 [0 0 1 1 0 0 1 1 0 0 0 1 1 1 0 1 0 0 0 0 1 0 1 0 0 0 0 3 1 1 0 1 1 0 0]
 [0 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]]


,&,& roll,actor,actor in,beauti,beauti citi,best,best actor,citi,good,...,teacher,teacher rock,the,the best,the rock,the teacher,the world,world,york,york is
0,1,1,0,0,0,0,0,0,0,1,...,1,1,1,0,0,1,0,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,0,0,3,1,1,0,1,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [X] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [X] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [13]:
column_transformer = ColumnTransformer(
        transformers=[
            ('bow', CountVectorizer(tokenizer= StemmerTokenizer(),ngram_range=(1,2)) , 'history_text'),
            ('minmax', MinMaxScaler() , ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score'])
        ])
pipeline_colum_transformer=Pipeline(steps=[('column_transformer', column_transformer)])
transformed_data=pipeline_colum_transformer.fit_transform(df_comics)
display(pd.DataFrame(transformed_data.toarray()))

,0,1,2,3,4,5,6,7,8,9,...,307707,307708,307709,307710,307711,307712,307713,307714,307715,307716
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.85,0.3,0.60,0.60,0.40,0.70
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.80,1.0,0.80,1.00,1.00,0.80
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.80,0.5,0.55,0.45,1.00,0.55
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.80,0.1,0.25,0.40,0.30,0.50
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.90,0.4,0.45,0.55,0.55,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.90,0.1,0.25,0.30,1.00,0.55
1281,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.80,1.0,1.00,1.00,1.00,0.80
1282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.95,0.5,1.00,0.75,1.00,0.80
1283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.75,0.1,1.00,0.30,1.00,0.30


In [8]:
#generamos el bag of words para cada fila
df_comics
bow = CountVectorizer(tokenizer= StemmerTokenizer(),ngram_range=(1,2))
bags_of_words = np.array(bow.fit_transform(df_comics['history_text']).toarray())
bow_columns=bow.get_feature_names_out()
bow_df=pd.DataFrame(bags_of_words, columns=bow_columns)
display(bow_df)
#aplicamos minmaxscaler a la los atributos de interes
minmax_column_transformer = ColumnTransformer(
        transformers=[
            #('bow', CountVectorizer(tokenizer= StemmerTokenizer(),ngram_range=(1,2)) , ['history_text']),
            ('minmax', MinMaxScaler() , ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score'])
        ])
minmaxed_attributes = minmax_column_transformer.fit_transform(df_comics)
minmaxed_attributes_df = pd.DataFrame(minmaxed_attributes, columns=['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score'])
display(minmaxed_attributes_df)
#join columns of bow_df and minmaxed_attributes_df
df_transformed = pd.concat([bow_df, minmaxed_attributes_df], axis=1)
#le aregamos el alignment
df_transformed = pd.concat([df_transformed, df_comics[['alignment']]])
#verificamos que se hayan sumado bien las columnas
#assert df_transformed.shape[1] == bow_df.shape[1] + minmaxed_attributes_df.shape[1]
display(df_transformed)
del minmaxed_attributes

,!,! !,! '',! (,! ),"! ,",! --,! .,! ``,! alison,...,�kick� .,�kick� and,�polic,�polic man�,�white,�white room�,�ǣmortal,�ǣmortal instruments���,��wors,"��wors ,"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1281,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1282,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1283,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score
0,0.85,0.3,0.60,0.60,0.40,0.70
1,0.80,1.0,0.80,1.00,1.00,0.80
2,0.80,0.5,0.55,0.45,1.00,0.55
3,0.80,0.1,0.25,0.40,0.30,0.50
4,0.90,0.4,0.45,0.55,0.55,0.85
...,...,...,...,...,...,...
1280,0.90,0.1,0.25,0.30,1.00,0.55
1281,0.80,1.0,1.00,1.00,1.00,0.80
1282,0.95,0.5,1.00,0.75,1.00,0.80
1283,0.75,0.1,1.00,0.30,1.00,0.30


## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [14]:
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
#seleccionamos las mejores features usando un SelectPercentile y un clasificador MultinomialNB
selector = SelectPercentile(score_func=f_classif, percentile=90)
classificador = MultinomialNB()
# generamos un pipeline con el pipeline de la sección 1.1, selector y el clasificador
pipeline = Pipeline(steps=[('11pipeline',pipeline_colum_transformer),('selector', selector), ('classifier', classificador)])
#separamos df en entrenamiento y prueba
y=df_comics[['alignment']]
X=df_comics.drop(['alignment'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42,stratify=y)
#entrenamos el pipeline
pipeline.fit(X_train, y_train)
#evaluamos el pipeline usando un clasificatrion_report
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


/home/pablo/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

         Bad       0.69      0.10      0.18        86
        Good       0.60      0.99      0.74       148
     Neutral       0.00      0.00      0.00        23

    accuracy                           0.60       257
   macro avg       0.43      0.36      0.31       257
weighted avg       0.58      0.60      0.49       257



/home/pablo/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pablo/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pablo/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
#creamos un Dummyclasifier y comparamos resultados
from sklearn.dummy import DummyClassifier
dummy_classifier = DummyClassifier(strategy='stratified')
dummy_classifier.fit(X_train, y_train)
y_pred = dummy_classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.36      0.34      0.35        86
        Good       0.57      0.60      0.58       148
     Neutral       0.10      0.09      0.09        23

    accuracy                           0.47       257
   macro avg       0.34      0.34      0.34       257
weighted avg       0.46      0.47      0.46       257



```
    Notamos que el calsificador que hicimos es mejor que el Dummy en clasificar Bad y Good, sin embargo no calsifica bien los Neutral
```

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [16]:
#revisamos como se llaman lso parámetros
pipeline.get_params()

{'memory': None,
 'steps': [('11pipeline',
   Pipeline(steps=[('column_transformer',
                    ColumnTransformer(transformers=[('bow',
                                                     CountVectorizer(ngram_range=(1,
                                                                                  2),
                                                                     tokenizer=<__main__.StemmerTokenizer object at 0x7f79f8c48160>),
                                                     'history_text'),
                                                    ('minmax', MinMaxScaler(),
                                                     ['intelligence_score',
                                                      'strength_score',
                                                      'speed_score',
                                                      'durability_score',
                                                      'power_score',
                                         

In [17]:

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
#pipeline = Pipeline(steps=[('11_pipeline',pipeline_colum_transformer),('selector', selector), ('classifier', classificador)])

params = [
       #SVC
       {'classifier': [SVC(random_state=42)],
        '11pipeline__column_transformer__bow__ngram_range':[(1,1), (1,2), (1,3)],
        'selector__percentile': [20, 40, 60, 80],
       },
       # Regresion logistica
       {'classifier': [LogisticRegression(random_state=42)],
        '11pipeline__column_transformer__bow__ngram_range':[(1,1), (1,2), (1,3)],
        'selector__percentile': [20, 40, 60, 80],
       },
       # Decision Tree Classifier
       {'classifier': [DecisionTreeClassifier(random_state=42)],
        '11pipeline__column_transformer__bow__ngram_range':[(1,1), (1,2), (1,3)],
        'selector__percentile': [20, 40, 60, 80],
       }       
       ]


grid_searcher = HalvingGridSearchCV(pipeline,params,verbose=10,n_jobs=-1)
grid_searcher.fit(X_train, y_train)
y_pred = grid_searcher.predict(X_test)
print(classification_report(y_test, y_pred))

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 38
max_resources_: 1028
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 36
n_resources: 38
Fitting 5 folds for each of 36 candidates, totalling 180 fits


/home/pablo/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 1/5; 1/36] START 11pipeline__column_transformer__bow__ngram_range=(1, 1), classifier=SVC(random_state=42), selector__percentile=20
[CV 3/5; 1/36] START 11pipeline__column_transformer__bow__ngram_range=(1, 1), classifier=SVC(random_state=42), selector__percentile=20
[CV 5/5; 1/36] START 11pipeline__column_transformer__bow__ngram_range=(1, 1), classifier=SVC(random_state=42), selector__percentile=20
[CV 3/5; 2/36] START 11pipeline__column_transformer__bow__ngram_range=(1, 1), classifier=SVC(random_state=42), selector__percentile=40
[CV 4/5; 1/36] START 11pipeline__column_transformer__bow__ngram_range=(1, 1), classifier=SVC(random_state=42), selector__percentile=20
[CV 2/5; 2/36] START 11pipeline__column_transformer__bow__ngram_range=(1, 1), classifier=SVC(random_state=42), selector__percentile=40
[CV 1/5; 2/36] START 11pipeline__column_transformer__bow__ngram_range=(1, 1), classifier=SVC(random_state=42), selector__percentile=40
[CV 2/5; 1/36] START 11pipeline__column_transformer__bo

/home/pablo/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/pablo/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/pablo/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/pablo/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

[CV 1/5; 1/36] END 11pipeline__column_transformer__bow__ngram_range=(1, 1), classifier=SVC(random_state=42), selector__percentile=20;, score=(train=0.833, test=0.714) total time=   0.4s
[CV 4/5; 2/36] START 11pipeline__column_transformer__bow__ngram_range=(1, 1), classifier=SVC(random_state=42), selector__percentile=40
[CV 5/5; 1/36] END 11pipeline__column_transformer__bow__ngram_range=(1, 1), classifier=SVC(random_state=42), selector__percentile=20;, score=(train=0.833, test=0.286) total time=   0.5s
[CV 5/5; 2/36] START 11pipeline__column_transformer__bow__ngram_range=(1, 1), classifier=SVC(random_state=42), selector__percentile=40


...

[CV 3/5; 2/2] END 11pipeline__column_transformer__bow__ngram_range=(1, 2), classifier=LogisticRegression(random_state=42), selector__percentile=20;, score=(train=0.991, test=0.698) total time=  23.1s
[CV 4/5; 2/2] START 11pipeline__column_transformer__bow__ngram_range=(1, 2), classifier=LogisticRegression(random_state=42), selector__percentile=20
[CV 1/5; 2/2] END 11pipeline__column_transformer__bow__ngram_range=(1, 2), classifier=LogisticRegression(random_state=42), selector__percentile=20;, score=(train=0.994, test=0.712) total time=  25.3s
[CV 5/5; 2/2] START 11pipeline__column_transformer__bow__ngram_range=(1, 2), classifier=LogisticRegression(random_state=42), selector__percentile=20
[CV 3/5; 1/2] END 11pipeline__column_transformer__bow__ngram_range=(1, 2), classifier=SVC(random_state=42), selector__percentile=60;, score=(train=0.591, test=0.576) total time=  24.9s
[CV 2/5; 2/2] END 11pipeline__column_transformer__bow__ngram_range=(1, 2), classifier=LogisticRegression(random_state

/home/pablo/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/pablo/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/pablo/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ),

[CV 4/5; 2/2] END 11pipeline__column_transformer__bow__ngram_range=(1, 2), classifier=LogisticRegression(random_state=42), selector__percentile=20;, score=(train=0.994, test=0.681) total time=  13.2s
[CV 5/5; 2/2] END 11pipeline__column_transformer__bow__ngram_range=(1, 2), classifier=LogisticRegression(random_state=42), selector__percentile=20;, score=(train=0.996, test=0.627) total time=  13.3s


/home/pablo/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/pablo/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

         Bad       0.68      0.58      0.63        86
        Good       0.74      0.90      0.81       148
     Neutral       0.75      0.13      0.22        23

    accuracy                           0.72       257
   macro avg       0.72      0.54      0.55       257
weighted avg       0.72      0.72      0.70       257



In [18]:
#vemso qué modelo se ha elegido
grid_searcher.best_params_

{'11pipeline__column_transformer__bow__ngram_range': (1, 2),
 'classifier': LogisticRegression(random_state=42),
 'selector__percentile': 20}

```
    Usando grid search se encontró que el mejor modelo era el de la regresión logística usando ngram_range=(1,2) en el bag of words, y percentil 20 en el selector. Notar que se podría ser más exhaustivo y probar otros modelos y variar distintos parámetros de cualquier sector del pipeline.
```

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [24]:
#### Código aquí ####
df_to_predict=df_comics_no_label[df_comics_no_label['name'].isin(['Vergil', 'Gorilla Girl', 'Batcow'])]
display(df_to_predict)
#predecimos
print(grid_searcher.predict(df_to_predict))


,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
16,122,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,529,Gorilla Girl,Fahnbullah Eddy,Fahnbullah Eddy,7,A carnival performer with the ability to turn ...,Gorilla Girl can transform into a talking gori...,90,35,60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
78,1368,Vergil,Vergil Sparda,NaN,16,"Vergil, later also known as Nelo Angelo, is on...",NaN,90,75,95,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
83,122,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


['Good' 'Good' 'Good' 'Good']


    Según lo predicho, todos los de la listas son Good

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>